In [1]:
from glob import glob
import os
import torch
import torchmetrics
import nibabel as nib
import subprocess
import numpy as np
import pandas as pd
import monai

In [ ]:
atlas_mri = "dataset3/Atlas/P56_Atlas_128_norm_id.nii.gz"
atlas_mask = "dataset3/Atlas/P56_Annotation_128_norm_id_mask.nii.gz"
atlas_labels = "dataset3/Atlas/atlas_gin_map6.nii.gz"

raw_mris = sorted(glob(os.path.join('dataset3', 'GIN', 'MRI', '*_affine.nii.gz')))

ants_fwd_warps = sorted(glob(os.path.join('dataset3', 'GIN', 'MRI', "*_warp.nii.gz")))
deep1_fwd_warps = sorted(glob(os.path.join('output', 'GIN', 'scenario1', 'DeformableWarp', "*_warp.nii.gz")))
deep2_fwd_warps = sorted(glob(os.path.join('output', 'GIN', 'scenario2', 'DeformableWarp', "*_warp.nii.gz")))
deep3_fwd_warps = sorted(glob(os.path.join('output', 'GIN', 'scenario3', 'DeformableWarp', "*_warp.nii.gz")))
mean_fwd_warps = sorted(glob(os.path.join('dataset3', 'GIN_Mean', 'MRI', "*_warp.nii.gz")))

ants_bck_warps = sorted(glob(os.path.join('dataset3', 'GIN', 'MRI', "*_warp_inverse.nii.gz")))
deep1_bck_warps = sorted(glob(os.path.join('output', 'GIN', 'scenario1', 'DeformableWarp', "*_invwarp.nii.gz")))
deep2_bck_warps = sorted(glob(os.path.join('output', 'GIN', 'scenario2', 'DeformableWarp', "*_invwarp.nii.gz")))
deep3_bck_warps = sorted(glob(os.path.join('output', 'GIN', 'scenario3', 'DeformableWarp', "*_invwarp.nii.gz")))
mean_bck_warps = sorted(glob(os.path.join('dataset3', 'GIN_Mean', 'MRI', "*_warp_inverse.nii.gz")))

names = ["Raw", "Affine", "ANTS", "OP", "DL", "DL+OP", "ANTS-Mean"]

list_dict = []

for i in range(len(raw_mris)):  
    for j in range(len(raw_mris)):
        
        output = 'ij.nii.gz'
        command = ['antsApplyTransforms']
        command.extend(['-d', '3'])
        command.extend(['-i', raw_mris[i]])
        command.extend(['-r', raw_mris[i]])
        command.extend(['-o', '['+output+',1]'])
        command.extend(['-n', 'Linear'])
        if i != j:
            command.extend(['-t', ants_bck_warps[i]])
            command.extend(['-t', ants_fwd_warps[j]])
            command.extend(['-t', ants_bck_warps[j]])
            command.extend(['-t', ants_fwd_warps[i]])
        else:
            command.extend(['-t', ants_bck_warps[i]])
            command.extend(['-t', ants_fwd_warps[i]])
        subprocess.call(command) 
        warp = nib.load(output).get_fdata()
        ants_inv_loss = np.sum(np.abs(warp)**2)/(warp.size)
        
        output = 'ij.nii.gz'
        command = ['antsApplyTransforms']
        command.extend(['-d', '3'])
        command.extend(['-i', raw_mris[i]])
        command.extend(['-r', raw_mris[i]])
        command.extend(['-o', '['+output+',1]'])
        command.extend(['-n', 'Linear'])
        if i != j:
            command.extend(['-t', deep1_bck_warps[i]])
            command.extend(['-t', deep1_fwd_warps[j]])
            command.extend(['-t', deep1_bck_warps[j]])
            command.extend(['-t', deep1_fwd_warps[i]])
        else:
            command.extend(['-t', deep1_bck_warps[i]])
            command.extend(['-t', deep1_fwd_warps[i]])
        subprocess.call(command) 
        warp = nib.load(output).get_fdata()
        deep1_inv_loss = np.sum(np.abs(warp)**2)/(warp.size)
        
        output = 'ij.nii.gz'
        command = ['antsApplyTransforms']
        command.extend(['-d', '3'])
        command.extend(['-i', raw_mris[i]])
        command.extend(['-r', raw_mris[i]])
        command.extend(['-o', '['+output+',1]'])
        command.extend(['-n', 'Linear'])
        if i != j:
            command.extend(['-t', deep2_bck_warps[i]])
            command.extend(['-t', deep2_fwd_warps[j]])
            command.extend(['-t', deep2_bck_warps[j]])
            command.extend(['-t', deep2_fwd_warps[i]])
        else:
            command.extend(['-t', deep2_bck_warps[i]])
            command.extend(['-t', deep2_fwd_warps[i]])
        subprocess.call(command) 
        warp = nib.load(output).get_fdata()
        deep2_inv_loss = np.sum(np.abs(warp)**2)/(warp.size)        
        
        output = 'ij.nii.gz'
        command = ['antsApplyTransforms']
        command.extend(['-d', '3'])
        command.extend(['-i', raw_mris[i]])
        command.extend(['-r', raw_mris[i]])
        command.extend(['-o', '['+output+',1]'])
        command.extend(['-n', 'Linear'])
        if i != j:
            command.extend(['-t', deep3_bck_warps[i]])
            command.extend(['-t', deep3_fwd_warps[j]])
            command.extend(['-t', deep3_bck_warps[j]])
            command.extend(['-t', deep3_fwd_warps[i]])
        else:
            command.extend(['-t', deep3_bck_warps[i]])
            command.extend(['-t', deep3_fwd_warps[i]])
        subprocess.call(command) 
        warp = nib.load(output).get_fdata()
        deep3_inv_loss = np.sum(np.abs(warp)**2)/(warp.size)        
        
        output = 'ij.nii.gz'
        command = ['antsApplyTransforms']
        command.extend(['-d', '3'])
        command.extend(['-i', raw_mris[i]])
        command.extend(['-r', raw_mris[i]])
        command.extend(['-o', '['+output+',1]'])
        command.extend(['-n', 'Linear'])
        if i != j:
            command.extend(['-t', mean_bck_warps[i]])
            command.extend(['-t', mean_fwd_warps[j]])
            command.extend(['-t', mean_bck_warps[j]])
            command.extend(['-t', mean_fwd_warps[i]])
        else:
            command.extend(['-t', mean_bck_warps[i]])
            command.extend(['-t', mean_fwd_warps[i]])
        subprocess.call(command) 
        warp = nib.load(output).get_fdata()
        mean_inv_loss = np.sum(np.abs(warp)**2)/(warp.size)

        name = raw_mris[i].split('/')[-1].split('_')[0]    
        ants_dict = {"ants_invloss": ants_inv_loss}
        deep1_dict = {"deep1_invloss": deep1_inv_loss}
        deep2_dict = {"deep2_invloss": deep2_inv_loss}
        deep3_dict = {"deep3_invloss": deep3_inv_loss}
        mean_dict = {"antsmean_invloss": mean_inv_loss}
        ith_dict = {"name": name, "i": i, "j": j}
        ith_dict.update(ants_dict)
        ith_dict.update(deep1_dict)
        ith_dict.update(deep2_dict)
        ith_dict.update(deep3_dict)
        ith_dict.update(mean_dict)
        list_dict.append(ith_dict)
df = pd.DataFrame(list_dict)
df.to_csv("inv_results.csv")
